In [1]:
import pandas as pd
import numpy as np 
import requests
import json
from pprint import pprint
from config import key_dict
from datetime import datetime
import time
import gmaps

In [2]:
base_url = "http://api.openweathermap.org/data/2.5/air_pollution/history?"
key = key_dict['owm']
lat = 32.7157
lng = 117.1611
start = 1521356400 #sunday march 18th 2018
end = 1616050800 #sunday march 18th 2021

In [3]:
url = f"{base_url}lat={lat}&lon={lng}&start={start}&end={end}&appid={key}"

data = requests.get(url).json()


In [4]:
pprint(data)

{'coord': {'lat': 32.7157, 'lon': 117.1611},
 'list': [{'components': {'co': 1054.76,
                          'nh3': 0.18,
                          'no': 20.79,
                          'no2': 40.44,
                          'o3': 5.14,
                          'pm10': 134.02,
                          'pm2_5': 114.88,
                          'so2': 34.81},
           'dt': 1606266000,
           'main': {'aqi': 5}},
          {'components': {'co': 1081.47,
                          'nh3': 0.32,
                          'no': 23.92,
                          'no2': 41.13,
                          'o3': 5.99,
                          'pm10': 141.88,
                          'pm2_5': 120.87,
                          'so2': 41.01},
           'dt': 1606269600,
           'main': {'aqi': 5}},
          {'components': {'co': 1108.17,
                          'nh3': 0.71,
                          'no': 23.69,
                          'no2': 45.24,
                          'o

                          'no2': 56.21,
                          'o3': 0.5,
                          'pm10': 115.57,
                          'pm2_5': 85.29,
                          'so2': 25.99},
           'dt': 1606572000,
           'main': {'aqi': 5}},
          {'components': {'co': 1188.28,
                          'nh3': 8.04,
                          'no': 9.84,
                          'no2': 57.58,
                          'o3': 0.07,
                          'pm10': 132.26,
                          'pm2_5': 99.64,
                          'so2': 30.52},
           'dt': 1606575600,
           'main': {'aqi': 5}},
          {'components': {'co': 1255.04,
                          'nh3': 10.13,
                          'no': 12.74,
                          'no2': 59.63,
                          'o3': 0.01,
                          'pm10': 138.35,
                          'pm2_5': 107.6,
                          'so2': 34.33},
           'dt': 1606579200,
   

          {'components': {'co': 1214.98,
                          'nh3': 1.9,
                          'no': 67.95,
                          'no2': 80.88,
                          'o3': 4.2,
                          'pm10': 124.28,
                          'pm2_5': 98.44,
                          'so2': 52.45},
           'dt': 1606964400,
           'main': {'aqi': 5}},
          {'components': {'co': 1001.36,
                          'nh3': 0.95,
                          'no': 44.26,
                          'no2': 76.08,
                          'o3': 7.06,
                          'pm10': 85.27,
                          'pm2_5': 68.73,
                          'so2': 54.36},
           'dt': 1606968000,
           'main': {'aqi': 5}},
          {'components': {'co': 807.76,
                          'nh3': 0.73,
                          'no': 27.05,
                          'no2': 65.8,
                          'o3': 15.38,
                          'pm10': 62.23,


                          'no2': 71.97,
                          'o3': 0,
                          'pm10': 159.19,
                          'pm2_5': 109.7,
                          'so2': 45.78},
           'dt': 1607508000,
           'main': {'aqi': 5}},
          {'components': {'co': 1281.74,
                          'nh3': 5.7,
                          'no': 40.68,
                          'no2': 71.29,
                          'o3': 0,
                          'pm10': 179.16,
                          'pm2_5': 125.07,
                          'so2': 48.16},
           'dt': 1607511600,
           'main': {'aqi': 5}},
          {'components': {'co': 1455.31,
                          'nh3': 4.43,
                          'no': 50.07,
                          'no2': 71.29,
                          'o3': 0,
                          'pm10': 197.71,
                          'pm2_5': 140.44,
                          'so2': 46.25},
           'dt': 1607515200,
          

                          'pm2_5': 85.19,
                          'so2': 57.22},
           'dt': 1607979600,
           'main': {'aqi': 5}},
          {'components': {'co': 1081.47,
                          'nh3': 26.6,
                          'no': 5.25,
                          'no2': 60.32,
                          'o3': 0.11,
                          'pm10': 110.45,
                          'pm2_5': 82.25,
                          'so2': 54.84},
           'dt': 1607983200,
           'main': {'aqi': 5}},
          {'components': {'co': 1148.22,
                          'nh3': 26.09,
                          'no': 7.94,
                          'no2': 59.63,
                          'o3': 0.07,
                          'pm10': 122.43,
                          'pm2_5': 88.88,
                          'so2': 54.84},
           'dt': 1607986800,
           'main': {'aqi': 5}},
          {'components': {'co': 1295.09,
                          'nh3': 25.59,
          

...

                          'no': 0,
                          'no2': 6.77,
                          'o3': 89.41,
                          'pm10': 22.04,
                          'pm2_5': 13.13,
                          'so2': 8.58},
           'dt': 1614092400,
           'main': {'aqi': 2}},
          {'components': {'co': 310.42,
                          'nh3': 6.46,
                          'no': 0,
                          'no2': 6.34,
                          'o3': 87.98,
                          'pm10': 21.35,
                          'pm2_5': 13.48,
                          'so2': 7.63},
           'dt': 1614096000,
           'main': {'aqi': 2}},
          {'components': {'co': 310.42,
                          'nh3': 6.21,
                          'no': 0,
                          'no2': 6.17,
                          'o3': 85.83,
                          'pm10': 21.46,
                          'pm2_5': 14.1,
                          'so2': 7.21},
           'd

                          'so2': 24.08},
           'dt': 1614560400,
           'main': {'aqi': 4}},
          {'components': {'co': 614.17,
                          'nh3': 2.38,
                          'no': 4.41,
                          'no2': 50.04,
                          'o3': 0.04,
                          'pm10': 40.99,
                          'pm2_5': 36.86,
                          'so2': 27.66},
           'dt': 1614564000,
           'main': {'aqi': 4}},
          {'components': {'co': 660.9,
                          'nh3': 2.69,
                          'no': 8.38,
                          'no2': 53.47,
                          'o3': 0.01,
                          'pm10': 46.19,
                          'pm2_5': 41.6,
                          'so2': 33.85},
           'dt': 1614567600,
           'main': {'aqi': 4}},
          {'components': {'co': 700.95,
                          'nh3': 2.82,
                          'no': 11.62,
                      

          {'components': {'co': 480.65,
                          'nh3': 12.92,
                          'no': 0,
                          'no2': 31.87,
                          'o3': 47.92,
                          'pm10': 54.29,
                          'pm2_5': 43.15,
                          'so2': 32.42},
           'dt': 1615032000,
           'main': {'aqi': 4}},
          {'components': {'co': 473.98,
                          'nh3': 14.44,
                          'no': 0,
                          'no2': 30.85,
                          'o3': 50.07,
                          'pm10': 48.8,
                          'pm2_5': 38.4,
                          'so2': 32.9},
           'dt': 1615035600,
           'main': {'aqi': 4}},
          {'components': {'co': 453.95,
                          'nh3': 14.82,
                          'no': 0,
                          'no2': 28.45,
                          'o3': 52.93,
                          'pm10': 42.13,
          

                          'o3': 134.47,
                          'pm10': 103.94,
                          'pm2_5': 96.33,
                          'so2': 12.28},
           'dt': 1615539600,
           'main': {'aqi': 5}},
          {'components': {'co': 460.63,
                          'nh3': 8.87,
                          'no': 0.01,
                          'no2': 10.45,
                          'o3': 120.16,
                          'pm10': 104.18,
                          'pm2_5': 94.51,
                          'so2': 11.44},
           'dt': 1615543200,
           'main': {'aqi': 5}},
          {'components': {'co': 514.03,
                          'nh3': 14.69,
                          'no': 0,
                          'no2': 13.2,
                          'o3': 101.57,
                          'pm10': 108.83,
                          'pm2_5': 94.97,
                          'so2': 11.44},
           'dt': 1615546800,
           'main': {'aqi': 5}},
          {

                          'pm10': 58.45,
                          'pm2_5': 49.13,
                          'so2': 24.32},
           'dt': 1616047200,
           'main': {'aqi': 4}},
          {'components': {'co': 377.18,
                          'nh3': 7.92,
                          'no': 0.58,
                          'no2': 21.42,
                          'o3': 74.39,
                          'pm10': 54.13,
                          'pm2_5': 45.26,
                          'so2': 23.84},
           'dt': 1616050800,
           'main': {'aqi': 4}}]}


In [5]:
aqi_df = pd.DataFrame(columns = ['Date','Lat','Long','AQI','CO','NO','NO2','O3','SO2','PM2.5','PM10','NH3'])

for i in range(len(data['list'])):

    components = data['list'][i]['components']

    aqi_df = aqi_df.append({'Date':datetime.fromtimestamp(data['list'][i]['dt']),
                            'Lat':data['coord']['lat'], 'Long':data['coord']['lon'],
                          'AQI':data['list'][i]['main']['aqi'], 'CO':components['co'], 'NO':components['no'],
                          'NO2':components['no2'], 'O3':components['o3'], 
                           'SO2':components['so2'], 'PM2.5':components['pm2_5'], 'PM10':components['pm10'],
                           'NH3':components['nh3']}, ignore_index = True)

aqi_df.set_index('Date', inplace = True)
aqi_df




,Lat,Long,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
Date,,,,,,,,,,,
2020-11-24 17:00:00,32.7157,117.1611,5,1054.76,20.79,40.44,5.14,34.81,114.88,134.02,0.18
2020-11-24 18:00:00,32.7157,117.1611,5,1081.47,23.92,41.13,5.99,41.01,120.87,141.88,0.32
2020-11-24 19:00:00,32.7157,117.1611,5,1108.17,23.69,45.24,3.00,45.78,126.01,148.66,0.71
2020-11-24 20:00:00,32.7157,117.1611,5,801.09,9.95,50.72,2.17,51.02,89.91,103.12,0.05
2020-11-24 21:00:00,32.7157,117.1611,5,700.95,7.71,47.98,5.99,48.64,75.60,84.50,0.05
...,...,...,...,...,...,...,...,...,...,...,...
2021-03-17 20:00:00,32.7157,117.1611,5,514.03,1.06,27.42,52.93,26.94,74.60,87.75,7.22
2021-03-17 21:00:00,32.7157,117.1611,5,473.98,1.08,26.05,56.51,26.46,63.72,75.46,7.16
2021-03-17 22:00:00,32.7157,117.1611,5,427.25,0.79,23.65,65.09,25.51,53.71,63.58,6.78


In [6]:
gmaps.configure(api_key = key_dict['google'])

In [25]:
aqi_df.iloc[0,0:2].values

array([32.7157, 117.1611], dtype=object)

In [29]:
#now figure out timesieries heatmap
# don't run this until you can figure out WHAT calls the api key. If it's just the base map, you're fine.
# if it's the heatmap layer too, that's a problem. 



#I've turned this cell into text for safety purposes so I don't accidentally run up a huge number of api calls

"""
#update 1am 3/23/21, I believe the next form of action if I want to generate a heatmap that changes over time
is to plot heatmap style colors individually, rather than rely on the heatmap function. data would look like
(x, y, intensity) sbut only take in 1 set of values from the df, where we've already computed the relative intensity 
on a 0-1 scale or something like that. We can then create a range of colors, from green to red (there's probably a color 
pack for this), and plotthose colors individually through a loop 


fig = gmaps.figure()


location = aqi_df.iloc[0,0:2].values
aqi_layer = gmaps.heatmap_layer(location, weights = aqi_df.iloc[0,2],
                                max_intensity = aqi_df['AQI'].max(),
                               dissipating = False)
                               
                               #I believe the above doesn't work because it's looking for multiple locations and weights
                               
                               
fig.add_layer(aqi_layer)

fig
time.sleep(0.2)

"""


TypeError: object of type 'int' has no len()